# Rekurencyjne Sieci Neuronowe (RNN)

### Importy i Utilsy  (odpalić i schować )

In [1]:
# imports 
import torch
import os
import unicodedata
import string
import numpy as np
from typing import Tuple, Optional, List

from torch.nn.functional import cross_entropy

import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score

from torch.utils.data import Dataset, DataLoader

all_letters = string.ascii_letters
n_letters = len(all_letters)


class ListDataset(Dataset):
    
    def __init__(self, data, targets):
        
        self.data = data
        self.targets = targets
        
    def __getitem__(self, ind):
        
        return self.data[ind], self.targets[ind]
    
    def __len__(self):
        return len(self.targets)

    
def unicode_to__ascii(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
                                                                 and c in all_letters)
                   

def read_lines(filename: str) -> List[str]:
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to__ascii(line) for line in lines]


def letter_to_index(letter: str) -> int:
    return all_letters.find(letter)


def line_to_tensor(line: str) -> torch.Tensor:
    tensor = torch.zeros(len(line), n_letters)
    for i, letter in enumerate(line):
        tensor[i][letter_to_index(letter)] = 1
    return tensor

## Dane sekwencyjne

Modele, którymi zajmowaliśmy się wcześniej zakładały konkretny kształt danych. Dla przykładu klasyczna sieć neuronowa fully-connected dla MNISTa zakładała, że na wejściu dostanie wektory rozmiaru 784 - dla wektorów o innej wymiarowości i innych obiektów model zwyczajnie nie będzie działać.

Takie założenie bywa szczególnie niewygodne przy pracy z niektórymi typami danych, takimi jak:
* językiem naturalny (słowa czy zdania mają zadanej z góry liczby znaków)
* szeregi czasowe (dane giełdowe ciągną się właściwie w nieskończoność) 
* dźwięk (nagrania mogą być krótsze lub dłuższe).

Do rozwiązania tego problemu służą rekuencyjne sieci neuronowe (*recurrent neural networks, RNNs*), które zapamiętują swój stan z poprzedniej iteracji.

### Ładowanie danych
Poniższe dwie komórki ściągają dataset nazwisk z 18 różnych narodowości. Każda litera w danym nazwisku jest zamieniana na jej indeks z alfabetu w postaci kodowania "one-hot". Inaczej mówiąc, każde nazwisko jest binarną macierzą rozmiaru `len(name)` $\times$ `n_letters`. 

Dodatkowo, ponieważ ten dataset jest mocno niezbalansowany, użyjemy specjalnego samplera do losowania przykładów treningowych, tak aby do uczenia sieć widziała tyle samo przykładów z każdej klasy.

Ponieważ nazwiska mogą mieć różne długości będziemy rozważać `batch_size = 1` w tym notebooku (choć implementacje modeli powinny działać dla dowolnych wartości `batch_size`!)

In [2]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

--2021-01-20 18:05:18--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 99.86.35.58, 99.86.35.85, 99.86.35.44, ...
Connecting to download.pytorch.org (download.pytorch.org)|99.86.35.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  --.-KB/s    in 0.06s   

2021-01-20 18:05:18 (48.5 MB/s) - ‘data.zip’ saved [2882130/2882130]

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Italian.tx

In [3]:
# NOTE: you can change the seed or remove it completely if you like
torch.manual_seed(1337)

data_dir = 'data/names'

data = []
targets = [] 
label_to_idx = {}

# read each natonality file and process data 
for label, file_name in enumerate(os.listdir(data_dir)):
    
    label_to_idx[label] = file_name.split('.')[0].lower()
    
    names = read_lines(os.path.join(data_dir, file_name))
    data += [line_to_tensor(name) for name in names]
    targets += len(names) * [label]

# split into train and test indices
test_frac = 0.1
n_test = int(test_frac * len(targets))
test_ind = np.random.choice(len(targets), size=n_test, replace=False)
train_ind = np.setdiff1d(np.arange(len(targets)), test_ind)

targets = torch.tensor(targets)
train_targets = targets[train_ind]

# calculate weights for BalancedSampler
uni, counts = np.unique(train_targets, return_counts=True)
weight_per_class = len(targets) / counts
weight = [weight_per_class[c] for c in train_targets]
# preapre the sampler
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weight, num_samples=len(weight)) 

train_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in train_ind], targets=train_targets)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1, sampler=sampler)

test_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in test_ind], targets=targets[test_ind])
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

In [4]:
# check out the content of the dataset
for i, (x, y) in enumerate(train_loader):
    break

print("x.shape:", x.shape)
print("name: ", end="")
for letter_onehot in x[0]:
    print(all_letters[torch.argmax(letter_onehot)], end="")

print("\ny:", label_to_idx[y.item()])

x.shape: torch.Size([1, 7, 52])
name: Ramires
y: portuguese


## Zadanie 1. (2 pkt.)

Zaimplementuj "zwykłą" sieć rekurencyjną. 
![rnn](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

* W klasie `RNN` należy zainicjalizować potrzebne wagi oraz zaimplementować główną logikę dla pojedynczej chwili czasowej $x_t$
* Wyjście z sieci możemy mieć dowolny rozmiar, potrzebna jest również warstwa przekształacjąca stan ukryty na wyjście.
* W pętli uczenia należy dodać odpowiednie wywołanie sieci. HINT: pamiętać o iterowaniu po wymiarze "czasowym".
* Zalecane jest użycie aktywacji na warstwie liczącej reprezentacje `hidden` tak, aby nie "eksplodowała", np. `tanh`.


In [5]:
class RNN(torch.nn.Module):
    
    def __init__(self, 
                 input_size: int,
                 hidden_size: int, 
                 output_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        :param output_size: int
            Desired dimensionality of the output vector
        """
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.input_to_hidden = torch.nn.Linear(input_size, hidden_size, bias=True)
        
        self.hidden_to_output = torch.nn.Linear(hidden_size, output_size, bias=True)

        self.hidden_to_hidden = torch.nn.Linear(hidden_size, hidden_size, bias=True)
    
    # for the sake of simplicity a single forward will process only a single timestamp 
    def forward(self, 
                input: torch.tensor, 
                hidden: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        :param hidden: torch.tensor
            Representation of the memory of the RNN from previous timestep
            shape [batch_size, hidden_size]
        """
        
        combined = torch.cat([input, hidden], dim=1)
        z = self.input_to_hidden(input) + self.hidden_to_hidden(hidden)
        hidden = torch.tanh(z)
        output =  self.hidden_to_output(hidden)
        return output, hidden
    
    def init_hidden(self, batch_size: int) -> torch.Tensor:
        """
        Returns initial value for the hidden state
        """
        return torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda()

### Pętla uczenia

In [9]:
n_class = len(label_to_idx)

# initialize network and optimizer
rnn = RNN(n_letters, 256, n_class).cuda()
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.01)   

# we will train for only a single epoch 
epochs = 1


# main loop
for epoch in range(epochs):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):  
        
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        # get initial hidden state
        hidden = rnn.init_hidden(x.shape[0])
        
        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the 2nd, time dimensiotn

        seq_len = x.shape[1]
        for i in range(seq_len):
          output, hidden = rnn.forward(x[:, i, :], hidden)
            
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()  
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []
    

# evaluate on the test set
with torch.no_grad():
    ps = []
    ys = []
    correct = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.cuda()
        ys.append(y.numpy())

        hidden = rnn.init_hidden(x.shape[0])
        seq_len = x.shape[1]
 
        for i in range(seq_len):
          output, hidden = rnn.forward(x[:, i, :], hidden)

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.15, "You should get over 0.15 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 2.890
Epoch: 0 Progress:  0% Loss: 2.882
Epoch: 0 Progress:  0% Loss: 2.847
Epoch: 0 Progress:  0% Loss: 2.853
Epoch: 0 Progress:  0% Loss: 2.878
Epoch: 0 Progress:  0% Loss: 2.825
Epoch: 0 Progress:  0% Loss: 2.851
Epoch: 0 Progress:  0% Loss: 2.890
Epoch: 0 Progress:  0% Loss: 2.886
Epoch: 0 Progress:  0% Loss: 2.836
Epoch: 0 Progress:  0% Loss: 2.758
Epoch: 0 Progress:  0% Loss: 2.690
Epoch: 0 Progress:  0% Loss: 2.838
Epoch: 0 Progress:  0% Loss: 2.759
Epoch: 0 Progress:  0% Loss: 2.898
Epoch: 0 Progress:  0% Loss: 2.781
Epoch: 0 Progress:  0% Loss: 2.736
Epoch: 0 Progress:  0% Loss: 2.801
Epoch: 0 Progress:  0% Loss: 2.763
Epoch: 0 Progress:  0% Loss: 2.729
Epoch: 0 Progress:  0% Loss: 2.755
Epoch: 0 Progress:  0% Loss: 2.823
Epoch: 0 Progress:  0% Loss: 2.724
Epoch: 0 Progress:  0% Loss: 2.659
Epoch: 0 Progress:  0% Loss: 2.643
Epoch: 0 Progress:  0% Loss: 2.661
Epoch: 0 Progress:  0% Loss: 2.728
Epoch: 0 Progress:  0% Loss: 2.589
Epoch: 0 Progress:  

## Zadanie 2. (0.5 pkt.)
Zaimplementuj funkcje `predict`, która przyjmuje nazwisko w postaci stringa oraz model RNN i wypisuje 3 najlepsze predykcje narodowości dla tego nazwiska razem z ich logitami.

**Hint**: Przyda się tutaj jedna z funkcji z pierwszej komórki notebooka.

In [74]:
def predict(name: str, rnn: RNN):
    """Prints the name and model's top 3 predictions with scores"""
    rnn = rnn.cpu()
    name_as_tensor = line_to_tensor(name).unsqueeze(0)
    hidden = rnn.init_hidden(1).cpu()
    for i in range(name_as_tensor.shape[1]):
      output, hidden = rnn(name_as_tensor[:, i, :], hidden)
    print(output.topk(dim=1, k=3))


In [75]:
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]

for name in some_names:
    print(name)
    predict(name, rnn)

Satoshi
torch.return_types.topk(
values=tensor([[5.3271, 3.2230, 2.7378]], grad_fn=<TopkBackward>),
indices=tensor([[13,  3,  8]]))
Jackson
torch.return_types.topk(
values=tensor([[3.4177, 2.6420, 2.4263]], grad_fn=<TopkBackward>),
indices=tensor([[ 1, 16,  2]]))
Schmidhuber
torch.return_types.topk(
values=tensor([[3.2671, 2.5741, 2.5422]], grad_fn=<TopkBackward>),
indices=tensor([[12,  8,  9]]))
Hinton
torch.return_types.topk(
values=tensor([[3.0138, 2.3515, 2.2496]], grad_fn=<TopkBackward>),
indices=tensor([[16,  1,  6]]))
Kowalski
torch.return_types.topk(
values=tensor([[5.6923, 5.0600, 3.8081]], grad_fn=<TopkBackward>),
indices=tensor([[ 3, 13,  2]]))


## Zadanie 3 (4 pkt.)
Ostatnim zadaniem jest implementacji komórki i sieci LSTM. 

![lstm](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

* W klasie `LSTMCell` ma znaleźć się główna loginka LSTMa, czyli wszystkie wagi do stanów `hidden` i `cell` jak i bramek kontrolujących te stany. 
* W klasie `LSTM` powinno znaleźć się wywołanie komórki LSTM, HINT: poprzednio było w pętli uczenia, teraz przenisiemy to do klasy modelu.
* W pętli uczenia należy uzupełnić brakujące wywołania do uczenia i ewaluacji modelu.

Zdecydowanie polecam [materiały Chrisa Olaha](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) do zarówno zrozumienia jak i ściągi do wzorów.

Zadaniem jest osiągnięcie wartości `f1_score` lepszej niż na sieci RNN, przy prawidłowej implementacji nie powinno być z tym problemów używając podanych hiperparametrów. Dozwolona jest oczywiście zmiana `random seed`.

#### Komórka LSTM

In [67]:
class LSTMCell(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTMCell, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        # initialize LSTM weights 
        # NOTE: there are different approaches that are all correct 
        # (e.g. single matrix for all input opperations), you can pick
        # whichever you like for this task
    
        self.wf = torch.nn.Linear(input_size, hidden_size)
        self.uf = torch.nn.Linear(hidden_size, hidden_size, bias=False)
        self.wi = torch.nn.Linear(input_size, hidden_size)
        self.ui = torch.nn.Linear(hidden_size, hidden_size, bias=False)
        self.wo = torch.nn.Linear(input_size, hidden_size)
        self.uo = torch.nn.Linear(hidden_size, hidden_size, bias=False)
        self.wc = torch.nn.Linear(input_size, hidden_size)
        self.uc = torch.nn.Linear(hidden_size, hidden_size, bias=False)
        self.c = torch.zeros((hidden_size,)) #  Poczatkowy stan z szumem, inspirowane https://r2rt.com/non-zero-initial-states-for-recurrent-neural-networks.html

    def forward(self, 
                input: torch.tensor, 
                states: Tuple[torch.tensor, torch.tensor]) -> Tuple[torch.tensor, torch.tensor]:
        
        hidden, cell = states
        
        # Compute input, forget, and output gates
        # then compute new cell state and hidden state
        # see http://colah.github.io/posts/2015-08-Understanding-LSTMs/ 
        f = torch.sigmoid(self.wf(input) + self.uf(hidden))
        i = torch.sigmoid(self.wi(input) + self.ui(hidden))
        o = torch.sigmoid(self.wo(input) + self.uo(hidden))

        cell = f * cell + i * torch.tanh(self.wc(input) + self.uc(hidden))
        
        hidden = o * torch.tanh(cell)
        
        return hidden, cell

### Klasa modelu LSTM

In [68]:
class LSTM(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = LSTMCell(input_size=input_size, hidden_size=hidden_size)
        
    def forward(self, 
                input: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        Returns Tuple of two torch.tensors, both of shape [seq_len, batch_size, hidden_size]
        """
        
        batch_size = input.shape[0]
        
        hidden, cell = self.init_hidden_cell(batch_size)
        
        hiddens = []
        cells = []
        
        # this time we will process the whole sequence in the forward method
        # as oppose to the previous exercise, remember to loop over the timesteps
        
        time_steps = input.shape[1]
        
        for i in range(time_steps):
          hidden, cell = self.cell.forward(input[:,i,:], [hidden, cell])
          hiddens.append(hidden)
          cells.append(cell)
        
        return torch.cat(hiddens), torch.cat(cells)
    
    def init_hidden_cell(self, batch_size):
        """
        Returns initial value for the hidden and cell states
        """
        return (torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda(), 
                torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda())

### Pętla uczenia

In [69]:
from itertools import chain

# torch.manual_seed(1337)

# build data loaders
train_loader = DataLoader(train_dataset, batch_size=1, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=1)

# initialize the lstm with an additional cliassifier layer at the top
lstm = LSTM(input_size=len(all_letters), hidden_size=128).cuda()
clf = torch.nn.Linear(in_features=128, out_features=len(label_to_idx)).cuda()

# initialize a optimizer
params = chain(lstm.parameters(), clf.parameters())
optimizer = torch.optim.Adam(params, lr=0.01) 

# we will train for only a single epoch 
epoch = 1

# main loop
for epoch in range(epoch):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):   
        
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the sequence length here
        # don't forget about the classifier!
        
        cell, hidden = lstm.forward(x)
        output = clf(hidden[-1, :].unsqueeze(0))
        
        # calucate the loss
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()                                
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []

# evaluate on the test set
with torch.no_grad():
    
    ps = []
    ys = []
    for i, (x, y) in enumerate(test_loader): 
        
        x = x.cuda()
        ys.append(y.numpy())
        
        cell, hidden = lstm.forward(x)
        output = clf(hidden[-1, :].unsqueeze(0))

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.18, "You should get over 0.18 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 2.911
Epoch: 0 Progress:  6% Loss: 2.377
Epoch: 0 Progress: 11% Loss: 1.842
Epoch: 0 Progress: 17% Loss: 1.692
Epoch: 0 Progress: 22% Loss: 1.438
Epoch: 0 Progress: 28% Loss: 1.315
Epoch: 0 Progress: 33% Loss: 1.212
Epoch: 0 Progress: 39% Loss: 1.169
Epoch: 0 Progress: 44% Loss: 1.098
Epoch: 0 Progress: 50% Loss: 1.012
Epoch: 0 Progress: 55% Loss: 1.060
Epoch: 0 Progress: 61% Loss: 1.031
Epoch: 0 Progress: 66% Loss: 0.955
Epoch: 0 Progress: 72% Loss: 0.857
Epoch: 0 Progress: 77% Loss: 0.811
Epoch: 0 Progress: 83% Loss: 0.837
Epoch: 0 Progress: 89% Loss: 0.812
Epoch: 0 Progress: 94% Loss: 0.812
Epoch: 0 Progress: 100% Loss: 0.751
Final F1 score: 0.24


## Zadanie 4. (0.5 pkt.)
Zaimplementuj analogiczną do funkcji `predict` z zadania 2 dla modelu `lstm+clf`.


In [87]:
def predict_lstm(name: str, lstm: LSTM, clf: torch.nn.Module):
    """Prints the name and model's top 3 predictions with scores"""
    lstm = lstm.cuda()
    clf = clf.cuda()
    name_as_tensor = line_to_tensor(name).unsqueeze(0).cuda()
    output = clf(hidden[-1, :].unsqueeze(0))
    print(output.topk(dim=1, k=3))


In [88]:
# test your lstm predictor
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]
    
for name in some_names:
    print(name)
    predict_lstm(name, lstm, clf)

Satoshi
torch.return_types.topk(
values=tensor([[11.1878,  4.8150,  3.8416]], device='cuda:0', grad_fn=<TopkBackward>),
indices=tensor([[17, 14, 11]], device='cuda:0'))
Jackson
torch.return_types.topk(
values=tensor([[11.1878,  4.8150,  3.8416]], device='cuda:0', grad_fn=<TopkBackward>),
indices=tensor([[17, 14, 11]], device='cuda:0'))
Schmidhuber
torch.return_types.topk(
values=tensor([[11.1878,  4.8150,  3.8416]], device='cuda:0', grad_fn=<TopkBackward>),
indices=tensor([[17, 14, 11]], device='cuda:0'))
Hinton
torch.return_types.topk(
values=tensor([[11.1878,  4.8150,  3.8416]], device='cuda:0', grad_fn=<TopkBackward>),
indices=tensor([[17, 14, 11]], device='cuda:0'))
Kowalski
torch.return_types.topk(
values=tensor([[11.1878,  4.8150,  3.8416]], device='cuda:0', grad_fn=<TopkBackward>),
indices=tensor([[17, 14, 11]], device='cuda:0'))
